In [2]:
!pip install opencv-python


In [3]:
from PIL import Image

# Instead of uploading or showing any real image:
print("Example image placeholder (no real picture used)")

# Create a simple gray image placeholder (does NOT embed big data)
img = Image.new("RGB", (256, 256), "gray")


Example image placeholder (no real picture used)


In [4]:
from PIL import ImageEnhance, ImageFilter, ImageOps

def make_professional_variant(img):
    img2 = ImageOps.autocontrast(img)
    img2 = ImageEnhance.Brightness(img2).enhance(1.1)
    img2 = ImageEnhance.Contrast(img2).enhance(1.1)
    img2 = img2.filter(ImageFilter.SHARPEN)
    return img2

def make_flawed_variant(img):
    img2 = ImageEnhance.Brightness(img).enhance(0.6)
    img2 = ImageEnhance.Contrast(img2).enhance(0.8)
    img2 = img2.filter(ImageFilter.GaussianBlur(radius=2))
    return img2

img_prof = make_professional_variant(img)
img_flawed = make_flawed_variant(img)

print("Generated professional-like and flawed variants.")


Generated professional-like and flawed variants.


In [5]:
import numpy as np
import cv2

def brightness_score(img):
    gray = np.array(img.convert("L"))
    return gray.mean() / 255.0

def sharpness_score(img):
    arr = np.array(img)
    lap = cv2.Laplacian(arr, cv2.CV_64F).var()
    return min(lap / 1000.0, 1.0)

def background_simple_score(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 80, 180)
    density = edges.mean() / 255.0
    return 1 - min(density, 1.0)

def professionalism_score(img):
    b = brightness_score(img)
    s = sharpness_score(img)
    bg = background_simple_score(img)
    score = (0.4*b + 0.3*s + 0.3*bg)
    return min(score, 1.0), (b, s, bg)


In [6]:
for name, image in [("Original", img), ("Professional-like", img_prof), ("Flawed", img_flawed)]:
    final, (b, s, bg) = professionalism_score(image)
    print(f"=== {name} ===")
    print(f"Professionalism Score: {final:.3f}")
    print(f"Brightness: {b:.3f}")
    print(f"Sharpness: {s:.3f}")
    print(f"Background simplicity: {bg:.3f}")
    print()


=== Original ===
Professionalism Score: 0.501
Brightness: 0.502
Sharpness: 0.000
Background simplicity: 1.000

=== Professional-like ===
Professionalism Score: 0.520
Brightness: 0.549
Sharpness: 0.000
Background simplicity: 1.000

=== Flawed ===
Professionalism Score: 0.419
Brightness: 0.298
Sharpness: 0.000
Background simplicity: 1.000



In [7]:
def feedback(img):
    score, (b, s, bg) = professionalism_score(img)
    f = []
    if b < 0.4: f.append("Lighting too dark")
    if s < 0.4: f.append("Image is blurry")
    if bg < 0.4: f.append("Background is cluttered")
    if not f: f.append("Looks professional!")
    return f

print("Feedback for Original:", feedback(img))
print("Feedback for Professional-like:", feedback(img_prof))
print("Feedback for Flawed:", feedback(img_flawed))


Feedback for Original: ['Image is blurry']
Feedback for Professional-like: ['Image is blurry']
Feedback for Flawed: ['Lighting too dark', 'Image is blurry']
